In [ ]:
from opt_einsum.paths import branch_1
!apt-get update
!apt-get install graphviz -y

!pip install tensorflow
!pip install numpy
!pip install pandas

!pip install keras
!pip install scikit-learn
!pip install matplotlib
!pip install joblib
!pip install pyarrow
!pip install fastparquet
!pip install scipy
!pip install seaborn
!pip install tqdm
!pip install pydot
!pip install tensorflow-io
!pip install tensorflow-addons


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, MultiHeadAttention, Dropout, BatchNormalization, LayerNormalization, Input, Activation, Lambda, Bidirectional, Add, MaxPooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW
import json
from datetime import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import tensorflow_addons as tfa

folder_name = datetime.now().strftime("%Y-%m-%d_%H-%M")

random_state_value = None

In [ ]:
def get_season(date):
    month = date.month
    day = date.day
    if (month == 12 and day >= 21) or (month <= 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month <= 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month <= 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month <= 12 and day < 21):
        return 'Autumn'
    else:
        return 'Unknown'


def get_time_period(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


def add_time_features(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['timestamp'] = df['datetime'].astype(np.int64) // 10 ** 9
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
    df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['day_of_year'] = df['datetime'].dt.dayofyear
    df['week_of_year'] = df['datetime'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['datetime'].dt.quarter
    df['is_month_end'] = df['datetime'].dt.is_month_end.astype(int)
    df['is_quarter_end'] = df['datetime'].dt.is_quarter_end.astype(int)
    df['is_year_end'] = df['datetime'].dt.is_year_end.astype(int)
    df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
    df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
    df['day_of_year_sin'] = np.sin(df['day_of_year'] * (2 * np.pi / 365.25))
    df['day_of_year_cos'] = np.cos(df['day_of_year'] * (2 * np.pi / 365.25))
    df['season'] = df['datetime'].apply(get_season)
    df['time_period'] = df['hour'].apply(get_time_period)
    return df


def add_solar_features(df):
    # Features based only on radiation and other available variables
    df['solar_elevation'] = np.sin(df['day_of_year'] * (2 * np.pi / 365.25)) * np.sin(df['hour'] * (2 * np.pi / 24))

    # Radiation-specific features
    df['radiation_clearsky'] = df['solarradiation'] * (100 - df['cloudcover']) / 100

    # Temperature impact on theoretical efficiency
    df['temp_efficiency_factor'] = 1 - 0.004 * (df['temp'] - 25)  # Typical temperature coefficient

    # Combined features
    df['cloud_impact'] = df['cloudcover'] * df['solarradiation']
    df['visibility_radiation'] = df['visibility'] * df['solarradiation']
    df['clear_sky_index'] = (100 - df['cloudcover']) / 100
    df['temp_effect'] = df['temp'] - df['tempmin']

    return df

def add_solar_specific_features(df):
    # Solar position and theoretical calculations
    df['day_length'] = 12 + 3 * np.sin(2 * np.pi * (df['day_of_year'] - 81) / 365.25)
    df['solar_noon_distance'] = np.abs(12 - df['hour'])
    df['solar_potential'] = df['clear_sky_index'] * np.cos(df['solar_noon_distance'] * np.pi / 12)

    # Rolling features for radiation
    windows = [3, 6, 12]
    for w in windows:
        df[f'radiation_rolling_{w}h'] = df['solarradiation'].rolling(window=w).mean()

    # Theoretical radiation features
    df['theoretical_radiation'] = df['solarradiation'] / (df['clear_sky_index'] + 1e-6)

    return df

def add_radiation_energy_features(df):
    # Features based only on radiation
    windows = [3, 6, 12]
    for w in windows:
        # Radiation features
        df[f'radiation_rolling_mean_{w}h'] = df['solarradiation'].rolling(window=w).mean()
        df[f'radiation_rolling_std_{w}h'] = df['solarradiation'].rolling(window=w).std()

    # Daily aggregations for radiation
    df['radiation_daily_sum'] = df.groupby(df.index.date)['solarradiation'].transform('sum')
    df['radiation_daily_max'] = df.groupby(df.index.date)['solarradiation'].transform('max')

    # Lag features for radiation
    lags = [1, 2, 3, 6]
    for lag in lags:
        df[f'radiation_lag_{lag}h'] = df['solarradiation'].shift(lag)

    return df

def add_advanced_features(df):
    df = add_time_features(df)
    df = add_solar_features(df)
    df = add_solar_specific_features(df)
    df = add_radiation_energy_features(df)

    df = pd.get_dummies(df, columns=['season', 'time_period'])

    df['temp_humidity_interaction'] = df['temp'] * df['humidity'] / 100
    df['radiation_temp_interaction'] = df['solarradiation'] * df['temp_efficiency_factor']
    df['radiation_cloud_interaction'] = df['solarradiation'] * (1 - df['cloudcover']/100)

    # Theoretical maximum based on clear sky conditions
    df['theoretical_max_radiation'] = df['solarradiation'] / (df['clear_sky_index'] + 1e-6)

    return df


def prepare_advanced_data(df):
    # Apply feature engineering functions
    df = add_advanced_features(df)

    target_variables = ['solarradiation', 'solarenergy', 'uvindex']

    selected_features = [
        # Weather features
        'temp', 'humidity', 'cloudcover', 'visibility',
        'temp_effect',

        # Solar radiation features
        'solarradiation',
        'radiation_clearsky',
        'radiation_rolling_mean_3h',
        'radiation_rolling_mean_6h',
        'radiation_daily_sum',
        'radiation_daily_max',
        'radiation_lag_1h',
        'radiation_lag_3h',

        # Temperature efficiency
        'temp_efficiency_factor',

        # Time features
        'hour_sin', 'hour_cos',
        'month_sin', 'month_cos',
        'day_of_year_sin', 'day_of_year_cos',

        # Solar position and potential
        'solar_elevation',
        'solar_potential',
        'day_length',
        'solar_noon_distance',

        # Clear sky and theoretical features
        'clear_sky_index',
        'theoretical_max_radiation',

        # Interaction features
        'radiation_temp_interaction',
        'radiation_cloud_interaction',
        'temp_humidity_interaction',
        'visibility_radiation'
    ]

    # Add one-hot columns
    categorical_columns = [col for col in df.columns if col.startswith(('season_', 'time_period_'))]
    final_features = selected_features + categorical_columns

    # Dataset preparation
    df = df.sort_values('datetime')
    df.set_index('datetime', inplace=True)

    # Handle missing values
    for column in final_features + target_variables:
        df[column] = df[column].interpolate(method='time')
    df.fillna(0, inplace=True)

    # Temporal split
    data_after_2010 = df[df['year'] >= 2010].copy()
    data_before_2010 = df[df['year'] < 2010].copy()

    X = data_after_2010[final_features]
    y = data_after_2010['solarenergy']
    X_to_predict = data_before_2010[final_features]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=random_state_value)

    # Scaling
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_to_predict_scaled = scaler.transform(X_to_predict)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, final_features, X_to_predict_scaled


def create_sequence_data(X, sequence_length=24):
    """
    Converts data into sequences for LSTM input
    sequence_length represents how many previous hours to consider
    """
    sequences = []
    for i in range(len(X) - sequence_length + 1):
        sequences.append(X[i:i + sequence_length])
    return np.array(sequences)


def prepare_hybrid_data(df):
    X_train_scaled, X_test_scaled, y_train, y_test, scaler, features, X_to_predict_scaled = prepare_advanced_data(df)

    # Convert data into sequences
    sequence_length = 24  # 24 hours of historical data

    X_train_seq = create_sequence_data(X_train_scaled, sequence_length)
    X_test_seq = create_sequence_data(X_test_scaled, sequence_length)

    # Adjust y by removing the first (sequence_length-1) elements
    y_train = y_train[sequence_length - 1:]
    y_test = y_test[sequence_length - 1:]

    X_to_predict_seq = create_sequence_data(X_to_predict_scaled, sequence_length)

    return X_train_seq, X_test_seq, y_train, y_test, scaler, features, X_to_predict_seq

In [ ]:
def create_residual_lstm_layer(x, units, dropout_rate, l2_reg=0.01, return_sequences=True, survival_probability=0.8):
    """
    Creates a residual LSTM layer with bidirectional processing, normalization, and dropout
    Parameters:
        x: input tensor
        units: number of LSTM units
        dropout_rate: dropout probability
        l2_reg: L2 regularization factor
        return_sequences: whether to return sequences or just the final output
        survival_probability: probability of layer survival for stochastic depth
    """
    residual = x
    x = Bidirectional(LSTM(units, return_sequences=return_sequences, kernel_regularizer=regularizers.l2(l2_reg)))(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)

    if return_sequences:
        # Adjust residual dimensions if needed
        if int(residual.shape[-1]) != 2 * units:
            residual = Dense(2 * units, activation='linear')(residual)
        x = tfa.layers.StochasticDepth(survival_probability)([x, residual])
    return x

def attention_block(x, units, num_heads=8, survival_probability=0.8):
    """
    Creates an attention block with multi-head attention and layer normalization
    Parameters:
        x: input tensor
        units: dimensionality of the attention layer
        num_heads: number of attention heads
        survival_probability: probability of layer survival for stochastic depth
    """
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=units)(x, x)
    x = tfa.layers.StochasticDepth(survival_probability)([x, attention])
    x = LayerNormalization()(x)
    return x

def create_solarradiation_model(input_shape, folder_name, l2_lambda=0.005):
    """
    Creates a deep learning model for solar radiation prediction
    Parameters:
        input_shape: shape of input data
        folder_name: directory to save model architecture visualization
        l2_lambda: L2 regularization factor
    """
    inputs = Input(shape=input_shape)

    # Define progressive hyperparameters for model depth
    survival_probs = [0.9, 0.8, 0.7]  # Decreasing survival probabilities
    attention_survival_probs = [0.85, 0.75, 0.65]  # Decreasing attention survival probabilities
    lstm_units = [256, 128, 64]  # Decreasing LSTM units
    dropout_rates = [0.4, 0.3, 0.2]  # Decreasing dropout rates
    attention_heads = [32, 24, 16]  # Decreasing attention heads

    # Build LSTM layers with attention blocks
    x = inputs
    for i in range(3):
        # Add residual LSTM layer
        x = create_residual_lstm_layer(
            x,
            units=lstm_units[i],
            dropout_rate=dropout_rates[i],
            l2_reg=l2_lambda,
            return_sequences=True,
            survival_probability=survival_probs[i]
        )
        # Add attention block
        x = attention_block(
            x,
            units=lstm_units[i],
            num_heads=attention_heads[i],
            survival_probability=attention_survival_probs[i]
        )
        if i < 2:  # No pooling after last LSTM layer
            x = MaxPooling1D()(x)

    # Final LSTM layer for sequence aggregation
    x = create_residual_lstm_layer(
        x,
        units=32,
        dropout_rate=0.1,
        l2_reg=l2_lambda,
        return_sequences=False,
        survival_probability=0.6
    )

    # Dense layers for final prediction
    dense_units = [64, 32]
    dense_dropout = [0.2, 0.1]

    for units, dropout in zip(dense_units, dense_dropout):
        x = Dense(units, kernel_regularizer=regularizers.l2(l2_lambda))(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(dropout)(x)

    # Output layer with value clipping
    outputs = Dense(1)(x)
    outputs = Lambda(lambda x: tf.clip_by_value(x, 0, 1500))(outputs)

    model = Model(inputs=inputs, outputs=outputs, name="SolarRadiationModel")

    # Configure optimizer with weight decay
    optimizer = AdamW(
        learning_rate=0.0003,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        weight_decay=0.001
    )

    # Custom evaluation metrics
    def rmse(y_true, y_pred):
        """Root Mean Square Error"""
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

    def mape(y_true, y_pred):
        """Mean Absolute Percentage Error"""
        epsilon = 1e-7
        return tf.reduce_mean(tf.abs((y_true - y_pred) / (y_true + epsilon))) * 100

    # Combined loss function
    def hybrid_loss(y_true, y_pred):
        """Weighted combination of MSE and MAE"""
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        mae = tf.reduce_mean(tf.abs(y_true - y_pred))
        return 0.7 * mse + 0.3 * mae

    # Compile model with custom loss and metrics
    model.compile(
        optimizer=optimizer,
        loss=hybrid_loss,
        metrics=[
            'mae',
            rmse,
            mape
        ]
    )
    model.summary()

    # Save model architecture visualization
    plot_model(model,
               to_file=f'{folder_name}_model_architecture.png',
               show_shapes=True,
               show_layer_names=True,
               dpi=150,
               show_layer_activations=True)

    return model


def evaluate_solarenergy_predictions(y_true, y_pred, hour=None, folder_name=None):
    """
    Comprehensive evaluation of solar energy predictions with detailed analysis and visualizations

    Parameters:
    -----------
    y_true : array-like
        Actual solar energy values (kWh)
    y_pred : array-like
        Predicted solar energy values (kWh)
    hour : array-like, optional
        Array of hours corresponding to predictions, for temporal analysis
    folder_name : str, optional
        Folder to save analysis plots

    Returns:
    --------
    dict
        Dictionary containing all calculated metrics
    """
    import os
    from datetime import datetime
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix

    # Data conversion and preparation
    y_true = np.array(y_true).ravel()
    y_pred = np.array(y_pred).ravel()
    errors = y_pred - y_true

    # Basic metrics
    mae_raw = mean_absolute_error(y_true, y_pred)
    rmse_raw = np.sqrt(mean_squared_error(y_true, y_pred))
    r2_raw = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-7))) * 100

    # Accuracy for error margins
    within_5_percent = np.mean(np.abs((y_pred - y_true) / (y_true + 1e-7)) <= 0.05)
    within_10_percent = np.mean(np.abs((y_pred - y_true) / (y_true + 1e-7)) <= 0.10)
    within_20_percent = np.mean(np.abs((y_pred - y_true) / (y_true + 1e-7)) <= 0.20)

    # Define solar energy levels
    def get_energy_level(value):
        if value <= 0.5:
            return 'Very Low'
        elif value <= 2.0:
            return 'Low'
        elif value <= 4.0:
            return 'Moderate'
        elif value <= 6.0:
            return 'High'
        elif value <= 8.0:
            return 'Very High'
        else:
            return 'Extreme'

    # Calculate levels
    y_true_levels = [get_energy_level(v) for v in y_true]
    y_pred_levels = [get_energy_level(v) for v in y_pred]
    level_accuracy = np.mean([t == p for t, p in zip(y_true_levels, y_pred_levels)])

    # Print main metrics
    print("\nSolar Energy Prediction Metrics:")
    print("\nAbsolute Metrics:")
    print(f"MAE: {mae_raw:.4f} kWh")
    print(f"RMSE: {rmse_raw:.4f} kWh")
    print(f"R² Score: {r2_raw:.3f}")
    print(f"MAPE: {mape:.2f}%")

    print("\nPercentage Accuracy:")
    print(f"Within ±5%: {within_5_percent*100:.1f}%")
    print(f"Within ±10%: {within_10_percent*100:.1f}%")
    print(f"Within ±20%: {within_20_percent*100:.1f}%")

    print("\nLevel Accuracy:")
    print(f"Level Accuracy: {level_accuracy*100:.1f}%")

    # Confusion matrix for levels
    cm = confusion_matrix(y_true_levels, y_pred_levels)
    print("\nConfusion Matrix for Levels:")
    cm_df = pd.DataFrame(
        cm,
        columns=['Very Low', 'Low', 'Moderate', 'High', 'Very High', 'Extreme'],
        index=['Very Low', 'Low', 'Moderate', 'High', 'Very High', 'Extreme']
    )
    print(cm_df)

    # Analysis by time periods
    if hour is not None:
        day_periods = {
            'Morning (5-11)': (5, 11),
            'Noon (11-13)': (11, 13),
            'Afternoon (13-17)': (13, 17),
            'Evening (17-21)': (17, 21),
            'Night (21-5)': (21, 5)
        }

        print("\nAnalysis by Time Period:")
        for period, (start, end) in day_periods.items():
            if start < end:
                mask = (hour >= start) & (hour < end)
            else:
                mask = (hour >= start) | (hour < end)

            if np.any(mask):
                period_mae = mean_absolute_error(y_true[mask], y_pred[mask])
                period_mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + 1e-7))) * 100
                period_mean = np.mean(y_true[mask])
                print(f"\n{period}:")
                print(f"MAE: {period_mae:.4f} kWh")
                print(f"MAPE: {period_mape:.2f}%")
                print(f"Mean Energy: {period_mean:.4f} kWh")

    # Visualizations
    if folder_name is not None:
        try:
            os.makedirs(folder_name, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Figure 1: Main analysis
            plt.figure(figsize=(20, 15))

            # Plot 1: Scatter plot
            plt.subplot(3, 2, 1)
            plt.scatter(y_true, y_pred, alpha=0.5)
            plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
            plt.xlabel('Actual Energy (kWh)')
            plt.ylabel('Predicted Energy (kWh)')
            plt.title('Actual vs Predicted Values')
            plt.grid(True)

            # Plot 2: Absolute errors distribution
            plt.subplot(3, 2, 2)
            plt.hist(errors, bins=50, alpha=0.7)
            plt.xlabel('Prediction Error (kWh)')
            plt.ylabel('Frequency')
            plt.title('Error Distribution')
            plt.grid(True)

            # Plot 3: Percentage errors distribution
            plt.subplot(3, 2, 3)
            percentage_errors = ((y_pred - y_true) / (y_true + 1e-7)) * 100
            plt.hist(np.clip(percentage_errors, -100, 100), bins=50, alpha=0.7)
            plt.xlabel('Percentage Error (%)')
            plt.ylabel('Frequency')
            plt.title('Percentage Error Distribution')
            plt.grid(True)

            # Plot 4: Errors vs Actual Values
            plt.subplot(3, 2, 4)
            plt.scatter(y_true, errors, alpha=0.5)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Actual Energy (kWh)')
            plt.ylabel('Error (kWh)')
            plt.title('Errors vs Actual Values')
            plt.grid(True)

            # Plot 5: Box plot errors by level
            plt.subplot(3, 2, 5)
            sns.boxplot(x=[get_energy_level(v) for v in y_true], y=errors)
            plt.xticks(rotation=45)
            plt.xlabel('Energy Level')
            plt.ylabel('Error (kWh)')
            plt.title('Error Distribution by Level')

            # Plot 6: Confusion matrix
            plt.subplot(3, 2, 6)
            sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)

            plt.tight_layout()
            filename = os.path.join(folder_name, f'solar_energy_analysis_{timestamp}.png')
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"\nPlot saved as: {filename}")
            plt.close()

            # Additional plot for temporal analysis if hour is available
            if hour is not None:
                plt.figure(figsize=(15, 8))
                plt.scatter(hour, errors, alpha=0.5)
                plt.axhline(y=0, color='r', linestyle='--')
                plt.xlabel('Hour of Day')
                plt.ylabel('Error (kWh)')
                plt.title('Error Distribution by Hour of Day')
                plt.grid(True)

                filename = os.path.join(folder_name, f'hourly_error_analysis_{timestamp}.png')
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                plt.close()

        except Exception as e:
            print(f"\nError saving plots: {str(e)}")

    # Additional metrics
    print("\nError Statistics:")
    print(f"Mean errors: {np.mean(errors):.4f} kWh")
    print(f"Standard deviation of errors: {np.std(errors):.4f} kWh")
    print(f"Median error: {np.median(errors):.4f} kWh")
    print(f"95th percentile absolute error: {np.percentile(np.abs(errors), 95):.4f} kWh")

    print("\nProduction Statistics:")
    print(f"Mean actual energy: {np.mean(y_true):.4f} kWh")
    print(f"Mean predicted energy: {np.mean(y_pred):.4f} kWh")
    print(f"Maximum actual energy: {np.max(y_true):.4f} kWh")
    print(f"Maximum predicted energy: {np.max(y_pred):.4f} kWh")

    # Return metrics in structured format
    metrics = {
        'absolute': {
            'mae': mae_raw,
            'rmse': rmse_raw,
            'r2': r2_raw,
            'mape': mape
        },
        'percentage_accuracy': {
            'within_5_percent': within_5_percent,
            'within_10_percent': within_10_percent,
            'within_20_percent': within_20_percent
        },
        'categorical': {
            'level_accuracy': level_accuracy
        },
        'error_stats': {
            'mean': float(np.mean(errors)),
            'std': float(np.std(errors)),
            'median': float(np.median(errors)),
            'p95_abs': float(np.percentile(np.abs(errors), 95))
        },
        'production_stats': {
            'mean_true': float(np.mean(y_true)),
            'mean_pred': float(np.mean(y_pred)),
            'max_true': float(np.max(y_true)),
            'max_pred': float(np.max(y_pred))
        }
    }

    return metrics


def plot_training_history(history, folder_name=None):
    """
    Display and save loss and metrics plots during training

    Parameters:
    -----------
    history : tensorflow.keras.callbacks.History
        History object returned by model training
    folder_name : str
        Folder to save the plot
    """
    import os

    try:
        # Create figure
        plt.figure(figsize=(12, 4))

        # Loss Plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # MAE Plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='Training MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title('Model MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()

        if folder_name is not None:
            os.makedirs(folder_name, exist_ok=True)
            # Generate filename with timestamp
            filename = os.path.join(folder_name, 'training_history.png')

            # Save figure
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"\nTraining history plot saved as: {filename}")

        # Save numerical data in CSV format
        history_df = pd.DataFrame({
            'epoch': range(1, len(history.history['loss']) + 1),
            'training_loss': history.history['loss'],
            'validation_loss': history.history['val_loss'],
            'training_mae': history.history['mae'],
            'validation_mae': history.history['val_mae']
        })

        if folder_name is not None:
            csv_filename = os.path.join(folder_name, 'training_history.csv')
            history_df.to_csv(csv_filename, index=False)
            print(f"Training history data saved as: {csv_filename}")

        # Calculate and save final statistics
        final_stats = {
            'final_training_loss': history.history['loss'][-1],
            'final_validation_loss': history.history['val_loss'][-1],
            'final_training_mae': history.history['mae'][-1],
            'final_validation_mae': history.history['val_mae'][-1],
            'best_validation_loss': min(history.history['val_loss']),
            'best_validation_mae': min(history.history['val_mae']),
            'epochs': len(history.history['loss']),
        }

        if folder_name is not None:
            # Save statistics in JSON format
            stats_filename = os.path.join(folder_name, 'training_stats.json')
            with open(stats_filename, 'w') as f:
                json.dump(final_stats, f, indent=4)
            print(f"Final statistics saved as: {stats_filename}")

        # Print main statistics
        print("\nFinal Training Statistics:")
        print(f"Final Loss (train/val): {final_stats['final_training_loss']:.4f}/{final_stats['final_validation_loss']:.4f}")
        print(f"Final MAE (train/val): {final_stats['final_training_mae']:.4f}/{final_stats['final_validation_mae']:.4f}")
        print(f"Best validation loss: {final_stats['best_validation_loss']:.4f}")
        print(f"Best validation MAE: {final_stats['best_validation_mae']:.4f}")

        plt.show()

    except Exception as e:
        print(f"\nError creating or saving plots: {str(e)}")


def train_hybrid_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32, folder_name='solarradiation_index'):
    """
    Advanced training function for the hybrid solar energy index model with detailed monitoring
    and training management.

    Parameters:
    -----------
    model : keras.Model
        The compiled hybrid model
    X_train : numpy.ndarray
        Training data
    y_train : numpy.ndarray
        Training targets
    X_test : numpy.ndarray
        Validation data
    y_test : numpy.ndarray
        Validation targets
    epochs : int, optional
        Maximum number of training epochs
    batch_size : int, optional
        Batch size

    Returns:
    --------
    history : keras.callbacks.History
        Training history with all metrics
    """

    # Advanced training callbacks
    callbacks = [
        # Early Stopping
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            mode='min',
            verbose=1,
            min_delta=1e-4
        ),
        # ReduceLROnPlateau for MAE
        ReduceLROnPlateau(
            monitor='mae',
            factor=0.2,
            patience=5,
            verbose=1,
            mode='min',
            min_delta=1e-4,
            cooldown=3,
            min_lr=1e-7
        ),
        # ReduceLROnPlateau for loss
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            verbose=1,
            mode='min',
            min_delta=1e-4,
            cooldown=2,
            min_lr=1e-7
        ),
        # Model Checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'{folder_name}_best_model.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=False
        ),
        # TensorBoard
        tf.keras.callbacks.TensorBoard(
            log_dir=f'./logs_{folder_name}',
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch',
            profile_batch=0
        ),
        # Lambda Callback for solar radiation monitoring
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=lambda epoch, logs: (
                lambda y_pred: print(
                    f"\nEpoch {epoch + 1}:"
                    f"\nPredictions out of range (0-1500 W/m²): "
                    f"{np.sum((y_pred < 0) | (y_pred > 1500))}"
                    f"\nMAPE: {np.mean(np.abs((y_test - y_pred) / (y_test + 1e-7))) * 100:.2f}%"
                    f"\nPredictions within ±10%: "
                    f"{np.mean(np.abs((y_pred - y_test) / (y_test + 1e-7)) <= 0.10) * 100:.2f}%"
                )
            )(model.predict(X_test)) if epoch % 20 == 0 else None
        )
    ]

    try:
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1,
            shuffle=False,
            validation_freq=1,
        )

        # Post-training analysis
        print("\nTraining completed successfully!")

        # Final evaluation on test set
        test_loss, test_mae, test_mse = model.evaluate(X_test, y_test, verbose=0)
        print(f"\nFinal metrics on test set:")
        print(f"Loss: {test_loss:.4f}")
        print(f"MAE: {test_mae:.4f}")
        print(f"MSE: {test_mse:.4f}")

        # Prediction analysis
        predictions = model.predict(X_test)
        out_of_range = np.sum((predictions < 0) | (predictions > 11))
        print(f"\nPredictions out of range: {out_of_range} ({out_of_range / len(predictions) * 100:.2f}%)")

        plot_training_history(history, folder_name=folder_name)

        return history

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

    finally:
        # Memory cleanup
        tf.keras.backend.clear_session()


def integrate_predictions(df, predictions, sequence_length=24):
    """
    Integrates solar energy index predictions into the original dataset for data before 2010.

    Parameters:
    -----------
    df : pandas.DataFrame
        Original dataset
    predictions : numpy.ndarray
        Array of solar energy index predictions
    sequence_length : int
        Sequence length used for predictions

    Returns:
    --------
    pandas.DataFrame
        Updated dataset with solar energy index predictions
    """
    # Convert datetime to datetime format if not already
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Identify rows before 2010
    mask_pre_2010 = df['datetime'].dt.year < 2010

    # Create temporary DataFrame with predictions
    dates_pre_2010 = df[mask_pre_2010]['datetime'].iloc[sequence_length - 1:]
    predictions_df = pd.DataFrame({
        'datetime': dates_pre_2010,
        'solarenergy_predicted': predictions.flatten()
    })

    # Merge with original dataset
    df = df.merge(predictions_df, on='datetime', how='left')

    # Update solarenergy column where missing
    df['solarenergy'] = df['solarenergy'].fillna(df['solarenergy_predicted'])

    # Remove temporary column
    df = df.drop('solarenergy_predicted', axis=1)

    print(f"Added {len(predictions)} predictions to dataset")
    print(f"Rows with solar energy index after integration: {df['solarenergy'].notna().sum()}")

    return df


def train_solarenergy_bounded_model(df):
    """
    Training of model with specific constraints for solar energy index
    """
    print("Initializing solar energy index model training...")

    try:
        # Data preparation
        print("\n1. Preparing data...")
        X_train_seq, X_test_seq, y_train, y_test, scaler, features, X_to_predict_seq = prepare_hybrid_data(df)

        print(f"Training data shape: {X_train_seq.shape}")
        print(f"Test data shape: {X_test_seq.shape}")

        # Data quality verification
        if np.isnan(X_train_seq).any() or np.isnan(y_train).any():
            raise ValueError("Found NaN values in training data")

        # Model creation
        print("\n2. Creating model...")
        input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
        model = create_solarradiation_model(input_shape, folder_name)

        print("\n4. Starting training...")
        history = train_hybrid_model(
            model=model,
            X_train=X_train_seq,
            y_train=y_train,
            X_test=X_test_seq,
            y_test=y_test,
            epochs=100,
            batch_size=128,
            folder_name=folder_name
        )

        print("\n5. Generating predictions...")
        predictions = model.predict(X_test_seq)
        predictions = np.clip(predictions, 0, 11)

        print("\n6. Evaluating model...")
        metrics = evaluate_solarenergy_predictions(y_test, predictions, folder_name=folder_name)

        # Create results dictionary
        training_results = {
            'model_params': {
                'input_shape': input_shape,
                'n_features': len(features),
                'sequence_length': X_train_seq.shape[1]
            },
            'training_params': {
                'batch_size': 32,
                'total_epochs': len(history.history['loss']),
                'best_epoch': np.argmin(history.history['val_loss']) + 1,
            },
            'performance_metrics': {
                'final_loss': float(history.history['val_loss'][-1]),
                'final_mae': float(history.history['val_mae'][-1]),
                'best_val_loss': float(min(history.history['val_loss'])),
                'out_of_range_predictions': int(np.sum((predictions < 0) | (predictions > 11)))
            }
        }

        print("\n7. Predicting missing data results...")
        to_predict_predictions = model.predict(X_to_predict_seq)
        to_predict_predictions = np.clip(to_predict_predictions, 0, 11)

        print("\n8. Integrating predictions into original dataset...")
        df_updated = integrate_predictions(df.copy(), to_predict_predictions)

        df_updated.to_parquet('../../sources/weather_data_complete.parquet')

        # Add prediction statistics to training_results
        training_results['prediction_stats'] = {
            'n_predictions_added': len(to_predict_predictions),
            'mean_predicted_solarenergy': float(to_predict_predictions.mean()),
            'min_predicted_solarenergy': float(to_predict_predictions.min()),
            'max_predicted_solarenergy': float(to_predict_predictions.max()),
        }

        print("\nTraining completed successfully!")

        return model, scaler, features, history, predictions, y_test, metrics, training_results

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

    finally:
        # Memory cleanup
        tf.keras.backend.clear_session()

In [ ]:
df = pd.read_parquet('../../sources/weather_data_solarradiation.parquet')

model, scaler, features, history, predictions, y_test, metrics, training_results = train_solarenergy_bounded_model(df)

In [ ]:
def plot_error_analysis(y_true, y_pred, folder_name=None):
    """
    Function to visualize prediction error analysis

    Parameters:
    -----------
    y_true : array-like
        Actual values
    y_pred : array-like
        Predicted values
    folder_name : str, optional
        Folder to save plots. If None, plots won't be saved.
    """
    import os
    from datetime import datetime

    # Convert to 1D numpy arrays if necessary
    if isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.Series):
        y_pred = y_pred.values

    y_true = y_true.ravel()
    y_pred = y_pred.ravel()

    # Calculate errors
    errors = y_pred - y_true

    # Create main figure
    fig = plt.figure(figsize=(15, 5))

    # Plot 1: Error Distribution
    plt.subplot(1, 3, 1)
    plt.hist(errors, bins=50, alpha=0.7)
    plt.title('Prediction Error Distribution')
    plt.xlabel('Error')
    plt.ylabel('Frequency')

    # Plot 2: Actual vs Predicted
    plt.subplot(1, 3, 2)
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')

    # Plot 3: Errors vs Actual Values
    plt.subplot(1, 3, 3)
    plt.scatter(y_true, errors, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.title('Errors vs Actual Values')
    plt.xlabel('Actual Values')
    plt.ylabel('Error')

    plt.tight_layout()

    # Save plot if folder is specified
    if folder_name is not None:
        try:
            # Create folder if it doesn't exist
            os.makedirs(folder_name, exist_ok=True)

            # Generate filename with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = os.path.join(folder_name, f'error_analysis_{timestamp}.png')

            # Save figure
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"\nPlot saved as: {filename}")
        except Exception as e:
            print(f"\nError saving plot: {str(e)}")

    plt.show()

    # Print error statistics
    print("\nError Statistics:")
    print(f"MAE: {np.mean(np.abs(errors)):.4f}")
    print(f"MSE: {np.mean(errors ** 2):.4f}")
    print(f"RMSE: {np.sqrt(np.mean(errors ** 2)):.4f}")
    print(f"Mean errors: {np.mean(errors):.4f}")
    print(f"Std errors: {np.std(errors):.4f}")

    # Calculate percentage of errors within thresholds
    thresholds = [0.5, 1.0, 1.5, 2.0]
    for threshold in thresholds:
        within_threshold = np.mean(np.abs(errors) <= threshold) * 100
        print(f"Predictions within ±{threshold}: {within_threshold:.1f}%")


plot_error_analysis(y_test, predictions, folder_name=folder_name)